In [1]:
from utils import FishEyeImage, tt11, rot
from photutils.detection import find_peaks, DAOStarFinder
from photutils.aperture import CircularAperture
from astropy.coordinates import angular_separation, position_angle
from astropy.table import Table, vstack
from astropy.stats import sigma_clipped_stats
import numpy as np
import matplotlib.pyplot as plt

hips_star = Table.read('hip2.fits')

file = 'IMG_8814'

pic = FishEyeImage(file+'.jpg', file+'.CR3')

solution = pic.solve(solve_size=1200)

2024-05-30 09:00:38,798:tetra3.Tetra3-INFO: Loading database from: c:\Users\mlang\anaconda3\Lib\site-packages\tetra3\data\default_database.npz


{'RA': 118.41849225024492, 'Dec': 44.45471209203223, 'Roll': 317.497525846909, 'FOV': 28.40096774281282, 'distortion': -0.008960927971853215, 'RMSE': 208.0028699531976, 'Matches': 26, 'Prob': 1.0193527449677233e-48, 'epoch_equinox': 2000, 'epoch_proper_motion': 2023.0, 'T_solve': 51.93950003013015, 'T_extract': 58.45180002506822}


In [2]:
pic.detect_stars()

c:\Users\mlang\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


id,xcentroid,ycentroid,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag
int32,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
1,332.13854905991633,166.42510777178873,0.7486194439585704,0.1968928643625197,0.34009041867102824,49,0.0,23265.0,1.2224808901723017,-0.2181051973877408
2,119.48633423955624,436.53568066899004,0.5236217355169522,0.2838560701461812,0.3269609429560629,49,0.0,27597.333333333336,1.794839592989715,-0.6350641030486502
1,946.9771968143461,87.98961695223723,0.6384080815936516,0.1554785975208034,0.1092939610274868,49,0.0,10205.333333333332,1.4031039438789852,-0.36772461338866413
2,649.2383788213949,220.86559930658407,0.5435839220914808,0.46247035806069464,0.23879750486664736,49,0.0,8537.0,1.1669550227556227,-0.1676352940434364
3,877.8004907868017,452.37611463239546,0.7293944075152924,0.2839958354681002,0.24783735946712787,49,0.0,13487.666666666666,1.6743103567025857,-0.5595899090987995
1,1155.5914560120682,128.481227933846,0.5986523467892617,0.5733213293097336,0.34490734470550893,49,0.0,7525.000000000001,1.4685358400782775,-0.4172113752033793
2,1227.9689576185117,160.8478779805779,0.6799387588968698,0.46083814122928207,0.3036247367680485,49,0.0,15994.333333333334,2.92523687414647,-1.1654025982416303
3,1192.8481853489748,164.15880523581325,0.6479873530513774,0.32119471121927723,0.17808175925913294,49,0.0,6438.666666666667,1.2257019688833328,-0.22096220929761506
4,1317.8219685488593,185.48164369886777,0.6884130892069007,0.5706651832002679,0.19650529100860706,49,0.0,10697.666666666666,1.9695125943625913,-0.73589690579853


In [3]:
idx, d2d, d3d=pic.match()

TypeError: concatenate() got an unexpected keyword argument 'dtype'